In this notebook, I collected all 2.1 million tweets for my project. Note: You cannot use this code now to get these tweets, as this way of getting tweets is only valid for tweets less than 7 days old.

In [2]:
import pandas as pd
from pymongo import MongoClient
import cnfg
from __future__ import print_function
import requests
from requests_oauthlib import OAuth1
import tweepy
import sys
import jsonpickle
import os
import time

In [18]:
client = MongoClient()
c = client
db = client.twe
reddit = client.reddit
db1 = client.redo
newtwee = db1.tweetsfull

Code for using Tweepy modified from: https://www.karambelkar.info/2015/01/how-to-use-twitters-search-rest-api-most-effectively./

In [ ]:
auth = tweepy.AppAuthHandler("", "")

api = tweepy.API(auth, wait_on_rate_limit=True,
 wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

In [17]:
#Verifying the api is working
type(api.statuses_lookup([869756298295947264]))

tweepy.models.ResultSet

In [ ]:
#searchQuery = '#someHashtag'  # this is what we're searching for
searchQuery = ("Kathy Griffin OR @kathygriffin")
maxTweets = 10000000# Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits
fName = 'tweetids.txt' # We'll store the tweets in a text file.


# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
#sinceID and maxID are required if one is trying to get tweets from specific times
sinceId = 869332529836380160

# If results only below a specific ID are, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id =  869914980065214464  

tweetCount = 0

print("Downloading max {0} tweets".format(maxTweets))
with open(fName, 'w') as f:
    for x in listids[9653:]:
        try:
            new_tweets = api.statuses_lookup(x)
            
            if not new_tweets:
                print("No more tweets found")
                break
            for tweet in new_tweets:
                tweet = tweet._json
                twee = {}
                twee["id"] = tweet["id"]
                twee["retweet_count"] = tweet["retweet_count"]
                twee["retweeted"] = tweet["retweeted"]
                twee["source"] = tweet["source"]
                twee["friends_count"] = tweet["user"]["friends_count"]
                twee["entities"] = tweet["entities"]
                #tweets are going into my MongoDB database
                tweets_other.insert_one(twee)
                f.write(str(tweet["id"]) + " ")

            tweetCount += len(new_tweets)
            tcount +=1
            if tweetCount % 3000 ==0:
                print("Downloaded {0} tweets".format(tweetCount))

            max_id = new_tweets[-1].id
        except tweepy.TweepError as e:
            # Just exit if any error
            print("some error : " + str(e))
            break

print ("Downloaded {0} tweets, Saved to {1}".format(tweetCount, fName))


Let's look at some tweets from the database to make sure there are no issues

In [20]:

Tweets = pd.DataFrame(list(newtwee.find({"id": {"$lt": 869914980065214464}})))

Looks Good!

In [21]:
Tweets.head()

,_id,coordinates,created_at,entities,favorite_count,favorited,followers_counts,friends_count,geo,id,...,name,num_statuses,retweet_count,retweeted,screen_name,source,text,user_desc,user_loc,verified
0,5937710c1e931e15ce11f63c,None,Wed May 31 13:54:27 +0000 2017,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,False,335,199,None,869914978903576577,...,Miriam Conley,21276,173,False,ConleyMiriam,"<a href=""http://twitter.com/download/android"" ...",RT @TreyYingst: President Trump on the Kathy G...,"Christian mother, widowed conservative for Pre...",,False
1,5937710c1e931e15ce11f63d,None,Wed May 31 13:54:27 +0000 2017,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,False,590,975,None,869914978865881088,...,TruthNado,7753,381,False,truth_nado,"<a href=""http://twitter.com/#!/download/ipad"" ...",RT @NolteNC: Hitler's gunna Hitler...\n\nCNN G...,,,False
2,5937710c1e931e15ce11f63e,None,Wed May 31 13:54:27 +0000 2017,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,False,1219,1120,None,869914978475687936,...,Cara Covfefe,26659,3130,False,cdm5866,"<a href=""http://twitter.com/download/iphone"" r...",RT @The_Trump_Train: Kathy Griffin did not mis...,"Mom, Army Vet, Trump Deplorable, Constitution,...",West BY GOD Virginia!,False
3,5937710c1e931e15ce11f63f,None,Wed May 31 13:54:27 +0000 2017,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,False,683,958,None,869914978307891203,...,Peter Askush,19133,20,False,peteraskush,"<a href=""http://twitter.com/download/iphone"" r...",RT @SharonMcCutchan: Kathy Griffin put a face ...,I read because I don't know enough. I'm okay p...,State of quiet desperation.,False
4,5937710c1e931e15ce11f640,None,Wed May 31 13:54:27 +0000 2017,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,False,268,2056,None,869914978131877888,...,VAB WARRIOR,5300,0,False,LOBSERVATEUR55,"<a href=""http://twitter.com/download/iphone"" r...",@usnews that is for me much Worst that what Ka...,,,False
